In [1]:
import yt
import astropy.units as u
%matplotlib inline 
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import h5py
from glob import glob

In [2]:
unit_base = {
    "length": (1.0, "pc"),
    "velocity": (100, "cm/s"),
    "mass": (1.0, "Msun"),
}
data = yt.load("../data/snapshot_296.hdf5", unit_base=unit_base)

yt : [INFO     ] 2024-04-22 17:24:39,503 ComovingIntegrationOn != 1 or (not found and OmegaLambda is 0.0), so we are turning off Cosmology.
yt : [INFO     ] 2024-04-22 17:24:39,533 Parameters: current_time              = 0.0002463474080710203
yt : [INFO     ] 2024-04-22 17:24:39,534 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-04-22 17:24:39,534 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-04-22 17:24:39,536 Parameters: domain_right_edge         = [8.16 8.16 8.16]
yt : [INFO     ] 2024-04-22 17:24:39,536 Parameters: cosmological_simulation   = 0


In [9]:
def LocateStar(data):
    ad = data.all_data()
    MassStar = ad["PartType5", "Masses"].in_units('Msun')
    ParticleID = ad['PartType5', 'ParticleIDs']
    print(ParticleID.size)
    Coordinates = ad['PartType5', 'Coordinates'].in_units('pc')
    sort = np.argmax(MassStar)
    Center = Coordinates[sort]
    return Center

In [10]:
def Particle_Limits(data):
    ad = data.all_data()
    ParticleID =  ad['PartType5', 'ParticleIDs']
    range = (ParticleID < 1e6 )
    return range

In [8]:
P_range = Particle_Limits(data)

1871

In [6]:
dir = 2
Center = LocateStar(data)
left_corner = data.domain_left_edge.copy()
right_corner = data.domain_right_edge.copy()
depth = data.quan(.5, "pc")
left_corner[dir] = Center[dir] - 0.5 * depth
right_corner[dir] = Center[dir] + 0.5 * depth


region = data.box(left_corner, right_corner)
p = yt.ProjectionPlot(data, "z", ("PartType0", "HII"), width= depth, center = Center, weight_field = ('gas', 'density'))
p.set_zlim(("PartType0", "HII"), 1e-9, 1)
p.set_lim(('PartType0', 'ParticleIDs'), 1, 1e6)

p

yt : [INFO     ] 2024-04-22 17:24:39,916 xlim = 3.848877 4.348877
yt : [INFO     ] 2024-04-22 17:24:39,916 ylim = 3.725362 4.225362
yt : [INFO     ] 2024-04-22 17:24:39,917 xlim = 3.848877 4.348877
yt : [INFO     ] 2024-04-22 17:24:39,917 ylim = 3.725362 4.225362
yt : [INFO     ] 2024-04-22 17:24:39,919 Making a fixed resolution buffer of (('PartType0', 'HII')) 800 by 800


1871


yt : [INFO     ] 2024-04-22 17:24:48,801 Making a fixed resolution buffer of (('gas', 'density')) 800 by 800


AttributeError: 'AxisAlignedProjectionPlot' object has no attribute 'set_lim'

In [ ]:
slc = yt.ProjectionPlot(data, "z", ('PartType0', 'MolecularMassFraction'), width= depth, center = Center, weight_field = ('PartType0', 'temperature'))
slc

In [ ]:
slice = yt.ProjectionPlot(data, "z", ('gas', 'H_density'), width= depth, center = Center, weight_field = ('gas', 'temperature'))
slice